# Delta Lake Setup - Configuração Completa

Este notebook configura o Delta Lake com suporte completo ao S3A (MinIO).

## ⚠️ IMPORTANTE

**Execute TODAS as células em ordem!**

1. Primeira célula: Configura PYSPARK_SUBMIT_ARGS ANTES de importar Spark
2. Segunda célula: Para qualquer Spark Session existente
3. Terceira célula: Cria Spark Session com JARs corretos
4. Quarta célula: Executa setup do Delta Lake

In [ ]:
# CÉLULA 1: Limpar PYSPARK_SUBMIT_ARGS para evitar conflitos
# IMPORTANTE: Não configurar PYSPARK_SUBMIT_ARGS aqui - vamos usar apenas spark.jars.packages
import os
import sys

# Configurar paths
os.environ['PYSPARK_PYTHON'] = 'python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3'

# Limpar PYSPARK_SUBMIT_ARGS se existir para evitar conflitos
if 'PYSPARK_SUBMIT_ARGS' in os.environ:
    print("⚠️  Removendo PYSPARK_SUBMIT_ARGS existente para evitar conflitos")
    del os.environ['PYSPARK_SUBMIT_ARGS']

print("✅ Ambiente configurado para usar apenas spark.jars.packages")

In [ ]:
# CÉLULA 2: Configurar Java e parar qualquer Spark Session existente
import os

# Configurar JAVA_HOME explicitamente
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
print(f"✅ JAVA_HOME configurado: {os.environ['JAVA_HOME']}")

# Parar qualquer Spark Session existente
try:
    from pyspark.sql import SparkSession
    existing_spark = SparkSession.getActiveSession()
    if existing_spark:
        print("⚠️  Parando Spark Session existente...")
        existing_spark.stop()
        import time
        time.sleep(5)  # Aguardar encerramento completo
        print("✅ Spark Session parada")
    else:
        print("✅ Nenhuma Spark Session ativa")
except Exception as e:
    print(f"ℹ️  {e}")
    print("✅ Pronto para criar nova Spark Session")

In [ ]:
# CÉLULA 3: Criar Spark Session com JARs do S3A e Delta Lake
import os
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

# Configurações MinIO
MINIO_ENDPOINT = "ch8ai-minio.l6zv5a.easypanel.host"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "1q2w3e4r"
BUCKET_NAME = "govbr"

print("\n[INFO] Criando Spark Session com JARs do S3A e Delta Lake...")
print("[INFO] Isso pode demorar 2-5 minutos na primeira execução enquanto baixa os JARs...")

# JARs do S3A e Delta Lake
# IMPORTANTE: Usar versão compatível com Spark 4.0.1
s3a_packages = "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262"
delta_package = "io.delta:delta-spark_2.13:4.0.0"

try:
    builder = SparkSession.builder \
        .appName("GovBR Delta Lake") \
        .config("spark.jars.packages", f"{delta_package},{s3a_packages}") \
        .config("spark.driver.memory", "2g") \
        .config("spark.executor.memory", "2g") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.hadoop.fs.s3a.endpoint", f"https://{MINIO_ENDPOINT}") \
        .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY) \
        .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY) \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "true") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
        .config("spark.master", "local[*]") \
        .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true") \
        .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")
    
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    
    print("\n✅ Spark Session criada!")
    print(f"✅ Versão Spark: {spark.version}")
    
    # Verificar se os JARs estão no classpath
    try:
        spark_context = spark.sparkContext
        packages = spark_context.getConf().get("spark.jars.packages", "")
        if "hadoop-aws" in packages:
            print("\n✅ JARs do S3A encontrados no classpath!")
            print(f"   Packages: {packages[:200]}...")
        else:
            print("\n⚠️  JARs do S3A podem não estar no classpath")
            print(f"   Packages: {packages}")
    except Exception as e:
        print(f"\n⚠️  Erro ao verificar classpath: {e}")
    
    import time
    print("\n[INFO] Aguardando carregamento completo dos JARs...")
    time.sleep(3)
    print("✅ Pronto para usar Delta Lake!")
    
except Exception as e:
    print(f"\n❌ Erro ao criar Spark Session: {e}")
    import traceback
    traceback.print_exc()
    raise

In [ ]:
# CÉLULA 4: Executar setup do Delta Lake
# Este script converte os dados Parquet em Delta Lake
print("\n" + "=" * 80)
print("EXECUTANDO SETUP DO DELTA LAKE")
print("=" * 80)

# Executar o script de setup
exec(open('delta_setup_spark.py').read())